<a href="https://colab.research.google.com/github/Dr-Abeera-Ramla-Javaid/Langchain_work/blob/main/Student_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade torch
!pip install python-dateutil


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
from transformers import pipeline
from datetime import datetime
from dateutil import parser
import random

# Initialize the conversational pipeline from Hugging Face
try:
    chatbot = pipeline("conversational", model="facebook/blenderbot-400M-distill")
except KeyError as e:
    print(f"Error initializing pipeline: {e}")
    print("Please ensure you have the latest version of the transformers library.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Error initializing pipeline: "Unknown task conversational, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"
Please ensure you have the latest version of the transformers library.


In [ ]:
def get_study_plan(subjects, hours_per_day, exam_dates):
    plan = {}
    for subject in subjects:
        # Parse the exam date
        try:
            exam_date = datetime.strptime(exam_dates[subject], "%Y-%m-%d")
        except ValueError:
            print(f"Invalid date format for {subject}. Skipping this subject.")
            continue

        days_left = (exam_date - datetime.now()).days
        if days_left <= 0:
            print(f"Exam date for {subject} has already passed or is today. Skipping this subject.")
            continue

        # Estimate study hours needed (customize as needed)
        hours_needed = random.randint(10, 15)  # Example estimation
        hours_per_day_subject = max(hours_needed // days_left, 1)  # At least 1 hour per day

        plan[subject] = {
            "exam_date": exam_date.strftime("%Y-%m-%d"),
            "total_hours_needed": hours_needed,
            "hours_per_day": hours_per_day_subject
        }
    return plan

def get_user_input():
    subjects = []
    exam_dates = {}

    # Collect subjects and exam dates
    print("Let's create your study plan!")
    while True:
        subject = input("Enter the subject name (or type 'done' to finish): ").strip()
        if subject.lower() == 'done':
            break
        if subject == '':
            print("Subject name cannot be empty. Please try again.")
            continue
        subjects.append(subject)
        while True:
            exam_date = input(f"Enter exam date for {subject} (e.g., YYYY-MM-DD, DD/MM/YYYY) or type 'skip' to skip this subject: ").strip()
            if exam_date.lower() == 'skip':
                print(f"Skipping {subject}.")
                subjects.pop()  # Remove the subject since it's being skipped
                break
            try:
                # Use dateutil to parse the date
                parsed_date = parser.parse(exam_date)
                # Convert to YYYY-MM-DD format
                formatted_date = parsed_date.strftime("%Y-%m-%d")
                exam_dates[subject] = formatted_date
                break
            except (ValueError, OverflowError):
                print("Invalid date format. Please enter the date in a recognizable format or type 'skip'.")

    # Collect available study hours per day
    while True:
        try:
            hours_per_day = int(input("How many hours can you study per day? "))
            if hours_per_day <= 0:
                print("Please enter a positive number.")
                continue
            break
        except ValueError:
            print("Please enter a valid number.")

    return subjects, exam_dates, hours_per_day


In [ ]:
def main():
    # Get the user's input (subjects, exam dates, hours per day)
    subjects, exam_dates, hours_per_day = get_user_input()

    if not subjects:
        print("No subjects entered. Exiting the study planner.")
        return

    # Generate the study plan based on the input
    study_plan = get_study_plan(subjects, hours_per_day, exam_dates)

    if not study_plan:
        print("No valid study plans could be generated.")
        return

    # Display the study plan
    print("\nHere is your personalized study plan:")
    for subject, details in study_plan.items():
        print(f"\nSubject: {subject}")
        print(f"  Exam Date: {details['exam_date']}")
        print(f"  Total Hours Needed: {details['total_hours_needed']} hours")
        print(f"  Hours to Study per Day: {details['hours_per_day']} hours")

    # Allow the user to ask additional questions
    while True:
        user_query = input("\nDo you have any other questions or need more help with your plan? (yes/no): ").strip().lower()
        if user_query == 'yes':
            user_input = input("Please type your question: ").strip()
            if user_input == '':
                print("You didn't type a question. Please try again.")
                continue
            try:
                response = chatbot(user_input)
                print(f"Chatbot says: {response[0]['generated_text']}")
            except Exception as e:
                print(f"Error generating response: {e}")
        elif user_query == 'no':
            print("Good luck with your studies! :)")
            break
        else:
            print("Please type 'yes' or 'no'.")


In [ ]:
if __name__ == "__main__":
    main()


Let's create your study plan!
Skipping english.
Skipping english.
Skipping kip.
